## 📓 04_chatgpt_claude_api.ipynb

# 🔌 ChatGPT og Claude API - Praktisk Integrasjon

## Læringsmål
- Sette opp og bruke OpenAI og Anthropic APIs
- Implementere feilhåndtering og rate limiting
- Bygge en enkel medisinsk chatbot

In [1]:
import os
from openai import OpenAI
from anthropic import Anthropic
from dotenv import load_dotenv
import time
from typing import Optional
import json

# Last miljøvariabler fra .env fil
load_dotenv()

print("🔌 API Integrasjon - Koble til ChatGPT og Claude")

🔌 API Integrasjon - Koble til ChatGPT og Claude


## Oppsett av API-klienter

In [2]:
class AIAssistant:
    """Wrapper-klasse for både OpenAI og Anthropic"""
    
    def __init__(self):
        # Initialiser klienter hvis API-nøkler finnes
        self.openai_client = None
        self.anthropic_client = None
        
        # Sjekk OpenAI
        if os.getenv("OPENAI_API_KEY"):
            try:
                self.openai_client = OpenAI()
                print("✅ OpenAI klient initialisert")
            except Exception as e:
                print(f"❌ OpenAI feil: {e}")
        else:
            print("⚠️ OpenAI API-nøkkel ikke funnet")
            
        # Sjekk Anthropic - prøv begge mulige navnene
        anthropic_key = os.getenv("ANTHROPIC_API_KEY") or os.getenv("CLAUDE_API_KEY")
        if anthropic_key:
            try:
                # Sett miljøvariabelen med riktig navn hvis nødvendig
                if not os.getenv("ANTHROPIC_API_KEY"):
                    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
                
                self.anthropic_client = Anthropic()
                print("✅ Anthropic klient initialisert")
            except Exception as e:
                print(f"❌ Anthropic feil: {e}")
        else:
            print("⚠️ Anthropic API-nøkkel ikke funnet (prøvde ANTHROPIC_API_KEY og CLAUDE_API_KEY)")
    
    def chat_gpt(self, prompt: str, model: str = "gpt-3.5-turbo", 
                 temperature: float = 0.7, max_tokens: int = 1000) -> Optional[str]:
        """Send prompt til ChatGPT"""
        if not self.openai_client:
            return "OpenAI ikke tilgjengelig - sjekk API-nøkkel"
        
        try:
            response = self.openai_client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"OpenAI feil: {e}"
    
    def chat_claude(self, prompt: str, model: str = "claude-3-5-sonnet-20240620",  # "claude-4-sonnet-20250514",
                   temperature: float = 0.7, max_tokens: int = 1000) -> Optional[str]:
        """Send prompt til Claude"""
        if not self.anthropic_client:
            return "Anthropic ikke tilgjengelig - sjekk API-nøkkel"
        
        try:
            response = self.anthropic_client.messages.create(
                model=model,
                max_tokens=max_tokens,
                temperature=temperature,
                messages=[{"role": "user", "content": prompt}]
            )
            return response.content[0].text
        except Exception as e:
            return f"Claude feil: {e}"
    
    def get_response(self, prompt: str, model_type: str = "gpt", **kwargs) -> str:
        """Enhetlig interface for begge modeller"""
        if model_type.lower() in ["gpt", "openai", "chatgpt"]:
            return self.chat_gpt(prompt, **kwargs)
        elif model_type.lower() in ["claude", "anthropic"]:
            return self.chat_claude(prompt, **kwargs)
        else:
            return f"Ukjent modelltype: {model_type}"
    
    def list_available_models(self):
        """Vis tilgjengelige modeller"""
        print("🤖 TILGJENGELIGE MODELLER:")
        print("-" * 40)
        
        if self.openai_client:
            print("OpenAI modeller:")
            openai_models = [
                "gpt-3.5-turbo",
                "gpt-4",
                "gpt-4-turbo", 
                "gpt-4o",
                "gpt-4o-mini"
            ]
            for model in openai_models:
                print(f"  • {model}")
        
        if self.anthropic_client:
            print("\nAnthropic modeller:")
            claude_models = [
                "claude-4-sonnet-20250514",    # Nyeste med god balanse mellom intelligens, hastighet og kostnad
                "claude-3-5-sonnet-20240620",  # Rask og intelligent
                "claude-3-5-haiku-20241022",   # Rask og billig
                "claude-3-opus-20240229",      # Kraftigst (men dyr)
            ]
            for model in claude_models:
                print(f"  • {model}")


In [3]:
# Debugging: Sjekk miljøvariabler
print("🔍 DEBUGGING MILJØVARIABLER:")
print("-" * 40)
print(f"OPENAI_API_KEY finnes: {'Ja' if os.getenv('OPENAI_API_KEY') else 'Nei'}")
print(f"ANTHROPIC_API_KEY finnes: {'Ja' if os.getenv('ANTHROPIC_API_KEY') else 'Nei'}")
print(f"CLAUDE_API_KEY finnes: {'Ja' if os.getenv('CLAUDE_API_KEY') else 'Nei'}")

# Vis de første og siste tegnene for sikkerhet
if os.getenv("OPENAI_API_KEY"):
    key = os.getenv("OPENAI_API_KEY")
    print(f"OpenAI nøkkel: {key[:7]}...{key[-4:]}")

anthropic_key = os.getenv("ANTHROPIC_API_KEY") or os.getenv("CLAUDE_API_KEY")
if anthropic_key:
    print(f"Anthropic nøkkel: {anthropic_key[:7]}...{anthropic_key[-4:]}")

print("\n🚀 INITIALISERER AI-ASSISTANT:")
print("-" * 40)

# Initialiser assistent
assistant = AIAssistant()

# Vis tilgjengelige modeller
assistant.list_available_models()


🔍 DEBUGGING MILJØVARIABLER:
----------------------------------------
OPENAI_API_KEY finnes: Ja
ANTHROPIC_API_KEY finnes: Nei
CLAUDE_API_KEY finnes: Ja
OpenAI nøkkel: sk-proj...ypoA
Anthropic nøkkel: sk-ant-...VQAA

🚀 INITIALISERER AI-ASSISTANT:
----------------------------------------
✅ OpenAI klient initialisert
✅ Anthropic klient initialisert
🤖 TILGJENGELIGE MODELLER:
----------------------------------------
OpenAI modeller:
  • gpt-3.5-turbo
  • gpt-4
  • gpt-4-turbo
  • gpt-4o
  • gpt-4o-mini

Anthropic modeller:
  • claude-4-sonnet-20250514
  • claude-3-5-sonnet-20240620
  • claude-3-5-haiku-20241022
  • claude-3-opus-20240229


In [4]:
# Test begge tjenester med nye modeller
print("\n🧪 TESTING TILKOBLINGER MED OPPDATERTE MODELLER:")
print("-" * 40)

test_prompt = "Si bare 'Hei' på norsk"

if assistant.openai_client:
    print("Testing OpenAI (GPT-3.5-turbo)...")
    gpt_response = assistant.chat_gpt(test_prompt)
    print(f"✅ GPT respons: {gpt_response}")

if assistant.anthropic_client:
    print("Testing Anthropic (claude-3-5-sonnet-20240620) ...")  # (claude-4-sonnet-20250514)...")
    claude_response = assistant.chat_claude(test_prompt)
    print(f"✅ Claude respons: {claude_response}")

# Test med ulike modeller
if assistant.openai_client and assistant.anthropic_client:
    print("\n🆚 MODELL-SAMMENLIGNING:")
    print("-" * 40)
    
    comparison_prompt = """Forklar kortfattet hva diabetes type 2 er til en pasient."""
    
    print("📝 Prompt:", comparison_prompt)
    print()
    
    # GPT-4o mini (rask og billig)
    print("🤖 GPT-4o-mini:")
    gpt_mini = assistant.chat_gpt(comparison_prompt, model="gpt-4o-mini", max_tokens=150)
    print(f"   {gpt_mini}")
    print()
    
    # Claude-3.5-Haiku (rask og billig)
    print("🎭 Claude-3.5-Haiku:")
    claude_haiku = assistant.chat_claude(comparison_prompt, model="claude-3-5-haiku-20241022", max_tokens=150)
    print(f"   {claude_haiku}")
    print()
    
    # claude-3-5-sonnet-20240620   # claude-4-sonnet-20250514 (balansert)
    print("🎨 claude-3-5-sonnet-20240620:")   # claude-4-sonnet-20250514:")
    claude_sonnet = assistant.chat_claude(comparison_prompt, max_tokens=150)
    print(f"   {claude_sonnet}")




🧪 TESTING TILKOBLINGER MED OPPDATERTE MODELLER:
----------------------------------------
Testing OpenAI (GPT-3.5-turbo)...
✅ GPT respons: Hei!
Testing Anthropic (claude-3-5-sonnet-20240620) ...


/var/folders/1g/_4x26lps6mgg1tr73d7g2bmh0000gp/T/ipykernel_65790/2147774612.py:58: DeprecationWarning: The model 'claude-3-5-sonnet-20240620' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = self.anthropic_client.messages.create(


✅ Claude respons: Hei

🆚 MODELL-SAMMENLIGNING:
----------------------------------------
📝 Prompt: Forklar kortfattet hva diabetes type 2 er til en pasient.

🤖 GPT-4o-mini:
   Diabetes type 2 er en tilstand hvor kroppen ikke bruker insulin effektivt, noe som fører til høye nivåer av sukker (glukose) i blodet. Insulin er et hormon som hjelper kroppen med å bruke sukker som energi. Over tid kan høye blodsukkernivåer skade organer og blodårer. 

Typiske symptomer inkluderer økt tørst, hyppig vannlating, tretthet og uskarpt syn. Behandling for diabetes type 2 inkluderer ofte livsstilsendringer som kosthold og trening, og noen ganger medisiner for å hjelpe med blodsukkerkontroll. Det er viktig å følge opp med legen for å håndtere

🎭 Claude-3.5-Haiku:
   Her er et forslag til hvordan du kan forklare diabetes type 2 enkelt og forståelig:

"Diabetes type 2 er en sykdom som handler om hvordan kroppen din håndterer sukker. Normalt tar hormonet insulin hånd om sukkeret i blodet og sørger for at ce

In [5]:
print("\n💡 MODELL-ANBEFALINGER:")
print("-" * 40)
print("🚀 For rask prototyping: gpt-4o-mini, claude-3-5-haiku")
print("⚖️ For balansert ytelse: gpt-4o, claude-4-sonnet-20250514")  
print("🧠 For komplekse oppgaver: gpt-4, claude-3-opus")
print("💰 Mest kostnadseffektiv: gpt-3.5-turbo, claude-3-5-haiku")


💡 MODELL-ANBEFALINGER:
----------------------------------------
🚀 For rask prototyping: gpt-4o-mini, claude-3-5-haiku
⚖️ For balansert ytelse: gpt-4o, claude-4-sonnet-20250514
🧠 For komplekse oppgaver: gpt-4, claude-3-opus
💰 Mest kostnadseffektiv: gpt-3.5-turbo, claude-3-5-haiku


## Sammenlign modeller på samme oppgave

In [6]:
# Medisinsk case for testing
medical_case = """
Forklar følgende blodprøveresultater for en pasient på en enkel måte:
- Hemoglobin: 10.2 g/dL (ref: 12-16)
- MCV: 72 fL (ref: 80-100)
- Ferritin: 8 ng/mL (ref: 15-200)

Hva kan dette indikere, og hva bør gjøres videre?
Svar på maks 100 ord, på norsk, tilpasset pasienten.
"""

print("🤖 ChatGPT svar:")
print("-" * 40)
gpt_response = assistant.chat_gpt(medical_case, temperature=0.3)
print(gpt_response)

print("\n🤖 Claude svar:")
print("-" * 40)
claude_response = assistant.chat_claude(medical_case, temperature=0.3)
print(claude_response)

🤖 ChatGPT svar:
----------------------------------------
Dine blodprøveresultater viser at du har lavt hemoglobin, lav MCV og lav ferritin. Dette kan tyde på jernmangelanemi, som kan skyldes for lite jern i kroppen. Du bør snakke med legen din for å få mer informasjon og eventuelt starte behandling med jerntilskudd og endre kostholdet ditt for å øke jerninntaket. Det er viktig å følge opp med jevnlige blodprøver for å sjekke om tilstanden bedrer seg.

🤖 Claude svar:
----------------------------------------


/var/folders/1g/_4x26lps6mgg1tr73d7g2bmh0000gp/T/ipykernel_65790/2147774612.py:58: DeprecationWarning: The model 'claude-3-5-sonnet-20240620' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = self.anthropic_client.messages.create(


Disse blodprøveresultatene tyder på at du kan ha jernmangelanemi. Det betyr at du har litt lavt jernnivå i kroppen, noe som påvirker produksjonen av røde blodceller.

Hemoglobin-verdien din er litt lav, noe som kan forklare hvorfor du kanskje føler deg trøtt eller svak. De lave MCV- og ferritin-verdiene støtter også at dette skyldes jernmangel.

Legen din vil sannsynligvis anbefale jerntilskudd for å øke jernnivået ditt. De kan også undersøke årsakene til jernmangelen, som kan være relatert til kosthold eller andre helseproblemer. 

Med riktig behandling kan disse verdiene normaliseres, og du vil sannsynligvis føle deg bedre.


## Bygge en medisinsk chatbot med kontekst

In [7]:
class MedicalChatbot:
    """Enkel medisinsk chatbot med samtalehistorikk"""
    
    def __init__(self, model_type: str = "openai"):
        self.model_type = model_type
        self.assistant = AIAssistant()
        self.conversation_history = []
        self.system_prompt = """
        Du er en hjelpsom medisinsk assistent for helsepersonell.
        Du gir informasjon basert på beste praksis, men minner om at 
        dine svar må valideres og ikke erstatter klinisk vurdering.
        Svar på norsk, vær presis og bruk korrekt medisinsk terminologi.
        """
    
    def add_message(self, role: str, content: str):
        """Legg til melding i samtalehistorikk"""
        self.conversation_history.append({"role": role, "content": content})
    
    def chat(self, user_input: str) -> str:
        """Håndter brukerinput og generer respons"""
        self.add_message("user", user_input)
        
        # Bygg full prompt med historikk
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.conversation_history[-10:])  # Maks 10 siste meldinger
        
        if self.model_type == "openai" and self.assistant.openai_client:
            try:
                response = self.assistant.openai_client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=messages,
                    temperature=0.7
                )
                bot_response = response.choices[0].message.content
            except Exception as e:
                bot_response = f"Feil: {e}"
        else:
            # Fallback til enkel prompt hvis OpenAI ikke tilgjengelig
            bot_response = "Simulert svar: Jeg forstår spørsmålet ditt om " + user_input[:50]
        
        self.add_message("assistant", bot_response)
        return bot_response
    
    def reset(self):
        """Nullstill samtalehistorikk"""
        self.conversation_history = []

In [8]:
# Test chatbot
chatbot = MedicalChatbot()

print("💬 Medisinsk Chatbot Demo")
print("=" * 50)

✅ OpenAI klient initialisert
✅ Anthropic klient initialisert
💬 Medisinsk Chatbot Demo


In [9]:
# Simuler en samtale
questions = [
    "Hva er normale verdier for blodtrykk?",
    "Hvilke faktorer kan påvirke disse verdiene?",
    "Hvordan behandles høyt blodtrykk?"
]

for q in questions:
    print(f"\n👤 Bruker: {q}")
    response = chatbot.chat(q)
    print(f"🤖 Bot: {response}")
    time.sleep(1)  # Unngå rate limiting


👤 Bruker: Hva er normale verdier for blodtrykk?
🤖 Bot: Normale verdier for blodtrykk varierer basert på alder, kjønn og andre faktorer. Generelt regnes et normalt blodtrykk å være under 120/80 mmHg. 

- Normalt blodtrykk: Under 120/80 mmHg
- Høyt normalt blodtrykk: 120-129/80-84 mmHg
- Hypertensjon (høyt blodtrykk): 130/80 mmHg og høyere

Det er viktig å merke seg at blodtrykksverdiene kan variere gjennom dagen og kan påvirkes av aktivitet, stress og andre faktorer. Hvis du har spørsmål om ditt blodtrykk, anbefales det å kontakte helsepersonell for videre oppfølging.

👤 Bruker: Hvilke faktorer kan påvirke disse verdiene?
🤖 Bot: Flere faktorer kan påvirke blodtrykksverdiene, inkludert:

1. Fysisk aktivitet: Trening eller fysisk anstrengelse kan midlertidig øke blodtrykket.
2. Stress og følelser: Sterke følelser som stress, angst eller sinne kan påvirke blodtrykket.
3. Koffein og tobakk: Inntak av koffeinholdige drikker eller røyking kan øke blodtrykket.
4. Kosthold: Et høyt saltinntak 

## Rate limiting og feilhåndtering

In [10]:
class RateLimitedAssistant:
    """API-klient med rate limiting og retry-logikk"""
    
    def __init__(self, requests_per_minute: int = 20):
        self.requests_per_minute = requests_per_minute
        self.min_time_between_requests = 60 / requests_per_minute
        self.last_request_time = 0
        self.client = OpenAI() if os.getenv("OPENAI_API_KEY") else None
    
    def wait_if_needed(self):
        """Vent hvis nødvendig for å respektere rate limits"""
        time_since_last = time.time() - self.last_request_time
        if time_since_last < self.min_time_between_requests:
            time.sleep(self.min_time_between_requests - time_since_last)
    
    def make_request(self, prompt: str, max_retries: int = 3) -> Optional[str]:
        """Gjør request med retry-logikk"""
        if not self.client:
            return "API ikke tilgjengelig"
        
        for attempt in range(max_retries):
            try:
                self.wait_if_needed()
                
                response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7
                )
                
                self.last_request_time = time.time()
                return response.choices[0].message.content
                
            except Exception as e:
                if attempt < max_retries - 1:
                    wait_time = 2 ** attempt  # Exponential backoff
                    print(f"Feil: {e}. Venter {wait_time} sekunder...")
                    time.sleep(wait_time)
                else:
                    return f"Feil etter {max_retries} forsøk: {e}"
        
        return None

In [11]:
# Test rate limiting
rate_limited = RateLimitedAssistant(requests_per_minute=20)

print("🔄 Tester rate-limited requests...")
for i in range(3):
    prompt = f"Gi meg en kort definisjon av: {['Anemi', 'Hypertensjon', 'Diabetes'][i]}"
    print(f"\nRequest {i+1}: {prompt[:30]}...")
    response = rate_limited.make_request(prompt)
    print(f"Respons: {response[:100]}..." if response else "Ingen respons")

🔄 Tester rate-limited requests...

Request 1: Gi meg en kort definisjon av: ...
Respons: Anemi er en tilstand karakterisert ved lavt antall røde blodlegemer eller lav hemoglobin i blodet, n...

Request 2: Gi meg en kort definisjon av: ...
Respons: Hypertensjon er en tilstand karakterisert ved høyt blodtrykk, der trykket i blodårene er høyere enn ...

Request 3: Gi meg en kort definisjon av: ...
Respons: Diabetes er en kronisk sykdom som påvirker kroppens evne til å regulere blodsukkernivået, enten på g...


## 💾 Lagre og gjenbruke responser

For å spare API-kostnader og forbedre ytelse, 
kan vi cache responser:

In [12]:
import hashlib
import json
from pathlib import Path
from datetime import datetime
from typing import Optional, Dict, Any

In [13]:
# Test caching-funksjonalitet
class CachedAssistant:
    """API-klient med caching av responser"""
    
    def __init__(self, cache_dir: str = "cache"):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.assistant = AIAssistant()
        self.cache_stats = {"hits": 0, "misses": 0}
    
    def get_cache_key(self, prompt: str, model: str, temperature: float = 0.7) -> str:
        """Generer unik cache-nøkkel for prompt"""
        content = f"{model}:{temperature}:{prompt}"
        return hashlib.md5(content.encode()).hexdigest()
    
    def get_cached_response(self, prompt: str, model: str, temperature: float = 0.7) -> Optional[str]:
        """Hent respons fra cache hvis den finnes"""
        cache_key = self.get_cache_key(prompt, model, temperature)
        cache_file = self.cache_dir / f"{cache_key}.json"
        
        if cache_file.exists():
            with open(cache_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f"📦 Hentet fra cache: {cache_key[:8]}... (spart API-kall)")
                self.cache_stats["hits"] += 1
                return data['response']
        
        self.cache_stats["misses"] += 1
        return None
    
    def save_to_cache(self, prompt: str, model: str, response: str, temperature: float = 0.7):
        """Lagre respons i cache"""
        cache_key = self.get_cache_key(prompt, model, temperature)
        cache_file = self.cache_dir / f"{cache_key}.json"
        
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump({
                'prompt': prompt,
                'model': model,
                'temperature': temperature,
                'response': response,
                'timestamp': datetime.now().isoformat(),
                'cache_key': cache_key
            }, f, ensure_ascii=False, indent=2)
        
        print(f"💾 Lagret i cache: {cache_key[:8]}...")
    
    def get_response(self, prompt: str, model: str = "gpt", temperature: float = 0.7) -> str:
        """Hent respons - først fra cache, så fra API"""
        # Prøv cache først
        cached_response = self.get_cached_response(prompt, model, temperature)
        if cached_response:
            return cached_response
        
        # Hvis ikke i cache, hent fra API
        print(f"🌐 Henter fra {model} API...")
        
        # Kall riktig metode basert på modelltype
        if model.lower() in ["gpt", "openai", "chatgpt"]:
            response = self.assistant.chat_gpt(prompt, temperature=temperature)
        elif model.lower() in ["claude", "anthropic"]:
            response = self.assistant.chat_claude(prompt, temperature=temperature)
        else:
            response = f"Ukjent modelltype: {model}"
        
        # Lagre i cache for fremtidig bruk
        self.save_to_cache(prompt, model, response, temperature)
        
        return response
    
    def get_cache_stats(self) -> Dict[str, Any]:
        """Vis cache-statistikk"""
        total = self.cache_stats["hits"] + self.cache_stats["misses"]
        hit_rate = self.cache_stats["hits"] / total * 100 if total > 0 else 0
        
        return {
            "cache_hits": self.cache_stats["hits"],
            "cache_misses": self.cache_stats["misses"],
            "hit_rate": f"{hit_rate:.1f}%",
            "total_requests": total,
            "cache_files": len(list(self.cache_dir.glob("*.json")))
        }
    
    def clear_cache(self):
        """Tøm cache"""
        for cache_file in self.cache_dir.glob("*.json"):
            cache_file.unlink()
        print(f"🗑️ Cache tømt - slettet {len(list(self.cache_dir.glob('*.json')))} filer")
        self.cache_stats = {"hits": 0, "misses": 0}

In [14]:
# Test caching-funksjonalitet
cached_assistant = CachedAssistant()

print("🧪 TESTING AV CACHE-FUNKSJONALITET")
print("=" * 50)

# Test-prompt som vi kjører flere ganger
test_prompt = """
Som erfaren allmennlege, gi en kort vurdering av denne pasienten:

Pasient: 65-årig mann med diabetes type 2
Symptomer: Tretthet, økt tørste, hyppig vannlating siste 2 uker
Blodprøver: HbA1c 9.2%, fastende glukose 15.2 mmol/L

Gi anbefaling for behandlingsopptrapping.
"""

✅ OpenAI klient initialisert
✅ Anthropic klient initialisert
🧪 TESTING AV CACHE-FUNKSJONALITET


In [15]:
print("🚀 Første kall (vil hente fra API):")
response1 = cached_assistant.get_response(test_prompt, "gpt", temperature=0.7)
print(f"Respons lengde: {len(response1)} tegn")

🚀 Første kall (vil hente fra API):
📦 Hentet fra cache: 4bf3aeae... (spart API-kall)
Respons lengde: 484 tegn


In [16]:
print("\n🔄 Andre kall (vil hente fra cache):")
response2 = cached_assistant.get_response(test_prompt, "gpt", temperature=0.7)
print(f"Respons lengde: {len(response2)} tegn")


🔄 Andre kall (vil hente fra cache):
📦 Hentet fra cache: 4bf3aeae... (spart API-kall)
Respons lengde: 484 tegn


In [17]:
print("\n📊 Cache-statistikk:")
stats = cached_assistant.get_cache_stats()
for key, value in stats.items():
    print(f"  {key}: {value}")


📊 Cache-statistikk:
  cache_hits: 2
  cache_misses: 0
  hit_rate: 100.0%
  total_requests: 2
  cache_files: 5


In [18]:
# Test med forskjellige modeller
print("\n\n🔄 SAMMENLIGNING MED CACHE")
print("=" * 50)

comparison_prompt = "Forklar diabetes type 2 på en enkel måte til en pasient"

print("📱 Testing begge modeller:")
for model in ["gpt", "claude"]:
    print(f"\n{model.upper()}:")
    response = cached_assistant.get_response(comparison_prompt, model, temperature=0.5)
    print(f"  Lengde: {len(response)} tegn")

print("\n📊 Oppdatert cache-statistikk:")
stats = cached_assistant.get_cache_stats()
for key, value in stats.items():
    print(f"  {key}: {value}")



🔄 SAMMENLIGNING MED CACHE
📱 Testing begge modeller:

GPT:
📦 Hentet fra cache: 112aef77... (spart API-kall)
  Lengde: 537 tegn

CLAUDE:
📦 Hentet fra cache: c38bba63... (spart API-kall)
  Lengde: 1160 tegn

📊 Oppdatert cache-statistikk:
  cache_hits: 4
  cache_misses: 0
  hit_rate: 100.0%
  total_requests: 4
  cache_files: 5


In [19]:
# Test deterministisk caching (temperature=0)
print("\n\n🔒 DETERMINISTISK CACHE-TEST (temperature=0)")
print("=" * 50)

deterministic_prompt = "List opp 3 hovedsymptomer på diabetes type 2"

print("Testing deterministisk respons (temperature=0):")
for i in range(3):
    print(f"\nKjøring {i+1}:")
    response = cached_assistant.get_response(deterministic_prompt, "gpt", temperature=0)
    print(f"  Respons: {response[:100]}...")

print("\n📊 Final cache-statistikk:")
stats = cached_assistant.get_cache_stats()
for key, value in stats.items():
    print(f"  {key}: {value}")



🔒 DETERMINISTISK CACHE-TEST (temperature=0)
Testing deterministisk respons (temperature=0):

Kjøring 1:
📦 Hentet fra cache: 5fa5e194... (spart API-kall)
  Respons: 1. Hyppig vannlating: Personer med diabetes type 2 kan oppleve økt behov for å urinere, spesielt om ...

Kjøring 2:
📦 Hentet fra cache: 5fa5e194... (spart API-kall)
  Respons: 1. Hyppig vannlating: Personer med diabetes type 2 kan oppleve økt behov for å urinere, spesielt om ...

Kjøring 3:
📦 Hentet fra cache: 5fa5e194... (spart API-kall)
  Respons: 1. Hyppig vannlating: Personer med diabetes type 2 kan oppleve økt behov for å urinere, spesielt om ...

📊 Final cache-statistikk:
  cache_hits: 7
  cache_misses: 0
  hit_rate: 100.0%
  total_requests: 7
  cache_files: 5


In [20]:
# Demonstrer forskjell mellom modeller
print("\n\n🔄 SAMMENLIGNING MED CACHE")
print("=" * 50)

comparison_prompt = "Forklar diabetes type 2 på en enkel måte til en pasient"

print("📱 Testing begge modeller:")
for model in ["gpt", "claude"]:
    print(f"\n{model.upper()}:")
    response = cached_assistant.get_response(comparison_prompt, model, temperature=0.5)
    print(f"  Lengde: {len(response)} tegn")

print("\n📊 Oppdatert cache-statistikk:")
stats = cached_assistant.get_cache_stats()
for key, value in stats.items():
    print(f"  {key}: {value}")



🔄 SAMMENLIGNING MED CACHE
📱 Testing begge modeller:

GPT:
📦 Hentet fra cache: 112aef77... (spart API-kall)
  Lengde: 537 tegn

CLAUDE:
📦 Hentet fra cache: c38bba63... (spart API-kall)
  Lengde: 1160 tegn

📊 Oppdatert cache-statistikk:
  cache_hits: 9
  cache_misses: 0
  hit_rate: 100.0%
  total_requests: 9
  cache_files: 5


In [21]:
# Cache-administrasjon
print("\n\n🛠️ CACHE-ADMINISTRASJON")
print("=" * 50)

def show_cache_contents():
    """Vis innhold i cache-mappen"""
    cache_files = list(cached_assistant.cache_dir.glob("*.json"))
    print(f"📁 Cache-mappe: {cached_assistant.cache_dir}")
    print(f"📄 Antall cache-filer: {len(cache_files)}")
    
    if cache_files:
        print("\n🔍 Cache-filer (første 3):")
        for i, cache_file in enumerate(cache_files[:3]):
            with open(cache_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f"  {i+1}. {cache_file.name}")
                print(f"     Model: {data['model']}")
                print(f"     Timestamp: {data['timestamp']}")
                print(f"     Prompt (start): {data['prompt'][:50]}...")
                print()

show_cache_contents()



🛠️ CACHE-ADMINISTRASJON
📁 Cache-mappe: cache
📄 Antall cache-filer: 5

🔍 Cache-filer (første 3):
  1. 112aef77b3ee7bfcd2caebb12a663f21.json
     Model: gpt
     Timestamp: 2025-09-10T01:31:49.564975
     Prompt (start): Forklar diabetes type 2 på en enkel måte til en pa...

  2. 5fa5e194ea221e15be9a4300a77d72e7.json
     Model: gpt
     Timestamp: 2025-09-10T01:32:05.053129
     Prompt (start): List opp 3 hovedsymptomer på diabetes type 2...

  3. 4bf3aeaef7a68aea009e5678fbf1bfa8.json
     Model: gpt
     Timestamp: 2025-09-10T01:31:11.554933
     Prompt (start): 
Som erfaren allmennlege, gi en kort vurdering av ...



In [22]:
# Cache-optimalisering tips
print("\n💡 CACHE-OPTIMALISERING TIPS")
print("=" * 50)

tips = [
    "✅ Bruk temperature=0 for deterministiske responser",
    "✅ Normaliser prompts (fjern unødvendige mellomrom)",
    "✅ Gruppe lignende spørsmål for bedre cache-utnyttelse",
    "✅ Overvåk cache hit rate - mål for >70%",
    "⚠️  Vær forsiktig med sensitive data i cache",
    "⚠️  Sett opp cache-utløp for data som blir utdatert",
    "💰 Cache kan spare betydelige API-kostnader"
]

for tip in tips:
    print(tip)



💡 CACHE-OPTIMALISERING TIPS
✅ Bruk temperature=0 for deterministiske responser
✅ Normaliser prompts (fjern unødvendige mellomrom)
✅ Gruppe lignende spørsmål for bedre cache-utnyttelse
✅ Overvåk cache hit rate - mål for >70%
⚠️  Vær forsiktig med sensitive data i cache
⚠️  Sett opp cache-utløp for data som blir utdatert
💰 Cache kan spare betydelige API-kostnader


In [23]:
# Avansert cache-funksjonalitet
class AdvancedCachedAssistant(CachedAssistant):
    """Utvidet cache med TTL og kategorisering"""
    
    def __init__(self, cache_dir: str = "cache", default_ttl_hours: int = 24):
        super().__init__(cache_dir)
        self.default_ttl_hours = default_ttl_hours
    
    def is_cache_expired(self, cache_file: Path, ttl_hours: int = None) -> bool:
        """Sjekk om cache er utløpt"""
        if ttl_hours is None:
            ttl_hours = self.default_ttl_hours
        
        with open(cache_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            cache_time = datetime.fromisoformat(data['timestamp'])
            age_hours = (datetime.now() - cache_time).total_seconds() / 3600
            return age_hours > ttl_hours
    
    def get_cached_response(self, prompt: str, model: str, temperature: float = 0.7, 
                          ttl_hours: int = None) -> Optional[str]:
        """Hent fra cache med TTL-sjekk"""
        cache_key = self.get_cache_key(prompt, model, temperature)
        cache_file = self.cache_dir / f"{cache_key}.json"
        
        if cache_file.exists():
            if self.is_cache_expired(cache_file, ttl_hours):
                cache_file.unlink()  # Slett utløpt cache
                print(f"⏰ Cache utløpt, slettet: {cache_key[:8]}...")
                self.cache_stats["misses"] += 1
                return None
            
            with open(cache_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                age_hours = (datetime.now() - datetime.fromisoformat(data['timestamp'])).total_seconds() / 3600
                print(f"📦 Fra cache ({age_hours:.1f}t gammel): {cache_key[:8]}...")
                self.cache_stats["hits"] += 1
                return data['response']
        
        self.cache_stats["misses"] += 1
        return None

In [24]:
# Test avansert cache
print("\n🚀 AVANSERT CACHE MED TTL")
print("=" * 50)

advanced_cache = AdvancedCachedAssistant(default_ttl_hours=1)

# Simuler cache med forskjellige aldre
test_prompt_ttl = "Hva er normal blodtrykk for en 50-åring?"

print("🕐 Test med TTL (Time To Live):")
response = advanced_cache.get_response(test_prompt_ttl, "gpt", temperature=0)
print(f"Respons: {response[:100]}...")

print(f"\n📊 Cache-statistikk:")
for key, value in advanced_cache.get_cache_stats().items():
    print(f"  {key}: {value}")


🚀 AVANSERT CACHE MED TTL
✅ OpenAI klient initialisert
✅ Anthropic klient initialisert
🕐 Test med TTL (Time To Live):
📦 Fra cache (0.0t gammel): 7f493b50...
Respons: Normalt blodtrykk for en 50-åring er vanligvis rundt 120/80 mmHg. Det anbefales imidlertjson at blod...

📊 Cache-statistikk:
  cache_hits: 1
  cache_misses: 0
  hit_rate: 100.0%
  total_requests: 1
  cache_files: 5


In [25]:
print("\n🎯 SAMMENDRAG: Cache-implementering")
print("=" * 50)
print("""
✅ FORDELER MED CACHING:
  • Reduserte API-kostnader (kan spare 50-90%)
  • Raskere responstid for gjentatte spørsmål  
  • Offline-tilgang til tidligere responser
  • Reproduserbare resultater for testing

⚠️  VIKTIGE HENSYN:
  • Sensitive data: Ikke cache personopplysninger
  • Utdatert informasjon: Bruk TTL for medisinske råd
  • Diskplass: Overvåk cache-størrelse
  • Konsistens: Cache kan skjule modell-oppdateringer

🏥 BRUK I HELSEVESENET:
  • Cache generelle medisinske spørsmål
  • Ikke cache pasient-spesifikke vurderinger
  • Sett kort TTL for behandlingsretningslinjer
  • Dokumenter hva som caches for compliance
""")


🎯 SAMMENDRAG: Cache-implementering

✅ FORDELER MED CACHING:
  • Reduserte API-kostnader (kan spare 50-90%)
  • Raskere responstid for gjentatte spørsmål  
  • Offline-tilgang til tidligere responser
  • Reproduserbare resultater for testing

⚠️  VIKTIGE HENSYN:
  • Sensitive data: Ikke cache personopplysninger
  • Utdatert informasjon: Bruk TTL for medisinske råd
  • Diskplass: Overvåk cache-størrelse
  • Konsistens: Cache kan skjule modell-oppdateringer

🏥 BRUK I HELSEVESENET:
  • Cache generelle medisinske spørsmål
  • Ikke cache pasient-spesifikke vurderinger
  • Sett kort TTL for behandlingsretningslinjer
  • Dokumenter hva som caches for compliance

